## Deliverable 2. Create a Customer Travel Destinations Map.

In [49]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [50]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Makamba,BI,-4.1348,29.8040,63.34,85,100,2.08,overcast clouds
1,1,Hermanus,ZA,-34.4187,19.2345,56.80,90,95,3.98,light rain
2,2,Dikson,RU,73.5069,80.5464,29.46,83,67,3.49,broken clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,65.01,74,89,4.88,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,79.29,76,75,8.05,light rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hilo,US,19.7297,-155.0900,79.29,76,75,8.05,light rain
6,6,Kapaa,US,22.0752,-159.3190,78.78,77,40,12.66,scattered clouds
8,8,Atambua,ID,-9.1061,124.8925,85.48,57,79,2.17,broken clouds
9,9,Nuevitas,CU,21.5453,-77.2644,77.18,78,9,11.74,clear sky
10,10,Ambon,ID,-3.6954,128.1814,89.74,62,40,4.61,scattered clouds
12,12,Makakilo City,US,21.3469,-158.0858,77.32,57,0,12.66,clear sky
14,14,Arlit,NE,18.7369,7.3853,84.52,25,46,7.02,scattered clouds
15,15,Mahebourg,MU,-20.4081,57.7000,79.09,69,40,8.05,scattered clouds
18,18,Vaini,TO,-21.2000,-175.2000,77.16,88,20,4.61,few clouds
22,22,Saint-Joseph,RE,-21.3667,55.6167,77.38,66,7,4.59,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [19]:
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hilo,US,19.7297,-155.0900,79.29,76,75,8.05,light rain
6,6,Kapaa,US,22.0752,-159.3190,78.78,77,40,12.66,scattered clouds
8,8,Atambua,ID,-9.1061,124.8925,85.48,57,79,2.17,broken clouds
9,9,Nuevitas,CU,21.5453,-77.2644,77.18,78,9,11.74,clear sky
10,10,Ambon,ID,-3.6954,128.1814,89.74,62,40,4.61,scattered clouds
12,12,Makakilo City,US,21.3469,-158.0858,77.32,57,0,12.66,clear sky
14,14,Arlit,NE,18.7369,7.3853,84.52,25,46,7.02,scattered clouds
15,15,Mahebourg,MU,-20.4081,57.7000,79.09,69,40,8.05,scattered clouds
18,18,Vaini,TO,-21.2000,-175.2000,77.16,88,20,4.61,few clouds
22,22,Saint-Joseph,RE,-21.3667,55.6167,77.38,66,7,4.59,clear sky


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hilo,US,79.29,light rain,19.7297,-155.0900,
6,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
8,Atambua,ID,85.48,broken clouds,-9.1061,124.8925,
9,Nuevitas,CU,77.18,clear sky,21.5453,-77.2644,
10,Ambon,ID,89.74,scattered clouds,-3.6954,128.1814,
12,Makakilo City,US,77.32,clear sky,21.3469,-158.0858,
14,Arlit,NE,84.52,scattered clouds,18.7369,7.3853,
15,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,
18,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
22,Saint-Joseph,RE,77.38,clear sky,-21.3667,55.6167,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             191
dtype: int64

In [38]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.count()

City                   191
Country                191
Max Temp               191
Current Description    191
Lat                    191
Lng                    191
Hotel Name             191
dtype: int64

In [39]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hilo,US,79.29,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Atambua,ID,85.48,broken clouds,-9.1061,124.8925,Kost Alfa Flat
9,Nuevitas,CU,77.18,clear sky,21.5453,-77.2644,Hotel Caonaba
10,Ambon,ID,89.74,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
12,Makakilo City,US,77.32,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
14,Arlit,NE,84.52,scattered clouds,18.7369,7.3853,Case de Passage De la Prefecture
15,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
18,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Saint-Joseph,RE,77.38,clear sky,-21.3667,55.6167,"""Plantation Bed and Breakfast"


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))